<a href="https://colab.research.google.com/github/ThisIsFarhan/gguf_conversion/blob/main/GGUF_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Downloading Fine-Tuned model

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!cp -r /content/drive/MyDrive/my_lora_finetuned_model.zip /content/


In [4]:
!unzip /content/my_lora_finetuned_model.zip -d /content/original_model/

Archive:  /content/my_lora_finetuned_model.zip
   creating: /content/original_model/content/my_lora_finetuned_model/
  inflating: /content/original_model/content/my_lora_finetuned_model/tokenizer.model  
  inflating: /content/original_model/content/my_lora_finetuned_model/config.json  
  inflating: /content/original_model/content/my_lora_finetuned_model/added_tokens.json  
  inflating: /content/original_model/content/my_lora_finetuned_model/tokenizer.json  
  inflating: /content/original_model/content/my_lora_finetuned_model/tokenizer_config.json  
  inflating: /content/original_model/content/my_lora_finetuned_model/generation_config.json  
  inflating: /content/original_model/content/my_lora_finetuned_model/model.safetensors  
  inflating: /content/original_model/content/my_lora_finetuned_model/special_tokens_map.json  


In [5]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 44430, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 44430 (delta 53), reused 29 (delta 28), pack-reused 44344 (from 4)
Receiving objects: 100% (44430/44430), 91.50 MiB | 28.02 MiB/s, done.
Resolving deltas: 100% (32013/32013), done.


In [38]:
!cd llama.cpp && rm -rf build && mkdir build && cd build && cmake .. -DGGML_CUDA=on && cmake --build . --config Release && cd .. && pip install -r requirements/requirements-convert_hf_to_gguf.txt

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP: TRUE (found version "4.5

In [13]:
!pip install -r /content/llama.cpp/requirements/requirements-convert_hf_to_gguf.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu124
    Uninstalling torch-2.5.1+cu124:
      Successfully uninstalled torch-2.5.1+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.20.1+cu124 requires torch==2.5.1, but you have torch 2.2.2+cpu which is incompatible.
torchaudio 2.5.1+cu124 requires torch==2.5.1, but you have torch 2.2.2+cpu which is incompatible.


In [22]:
!python ./llama.cpp/convert_hf_to_gguf.py /content/original_model/content/my_lora_finetuned_model --outfile f16.gguf --outtype f16

INFO:hf-to-gguf:Loading model: my_lora_finetuned_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:output.weight,               torch.float16 --> F16, shape = {2048, 32002}
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {2048, 32002}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> F16, shape = {5632, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {2048, 256}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> F16, shape = {20

In [28]:
!chmod +x /content/llama.cpp/examples/quantize/quantize.cpp

In [45]:
!/content/llama.cpp/build/bin/llama-quantize f16.gguf tinydolphin_FYP_q8_0.gguf Q8_0

main: build = 4747 (c5d91a74)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing 'f16.gguf' to 'tinydolphin_FYP_q8_0.gguf' as Q8_0
llama_model_loader: loaded meta data with 32 key-value pairs and 201 tensors from f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = TinyDolphin 2.8 1.1b
llama_model_loader: - kv   3:                       general.organization str              = Cognitivecomputations
llama_model_loader: - kv   4:                           general.basename str              = TinyDolphin-2.8
llama_model_loader: - kv   5:                         general.size_label str 

In [40]:
!pip install ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 63.4 MB/s eta 0:00:00


In [53]:
!pip install llama-cpp-python langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18


In [67]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/f16.gguf",
    n_gpu_layers=0,
    max_tokens=100,
    verbose=False
)

llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32
llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [68]:
user_input = "left_input: phone, chair, lamp\nright_input: cabinet, person, human\nup_input: mouse, glasses\nbottom_input: laptop, shelf"
template = f"<|user|>\n{user_input}\n<|assistant|>"
llm.invoke(template)

'\nThe left portion contains a phone, a chair, and a lamp. In the right area, there were a cabinet, a person, and a human. The upper portion contains a mouse and a glasses. The bottom portion contains a laptop and a shelf.<|person|>\n<|child|>\n<|assistant|>\nThe left portion contained a phone and a lamp. In the right area, there was a cabinet. The top portion contains a mouse'

In [69]:
!cp /content/tinydolphin_FYP_q5_0.gguf /content/drive/MyDrive/

In [70]:
!cp /content/tinydolphin_FYP_q8_0.gguf /content/drive/MyDrive/